In [3]:
import requests
import requests
import zipfile
from io import BytesIO
from glob import glob
from bs4 import BeautifulSoup
def download_and_extract_zip(url, extract_path="."):
    """
    Downloads a ZIP file from a given URL and extracts its contents.

    Args:
        url (str): The URL of the ZIP file to download.
        extract_path (str): The directory where the contents will be extracted.
                            Defaults to the current directory.
    """
    try:
        # Send a GET request to the URL to download the file content
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Use BytesIO to treat the downloaded content as a file-like object
        zip_file_in_memory = BytesIO(response.content)

        # Open the ZIP file from memory
        with zipfile.ZipFile(zip_file_in_memory, 'r') as zf:
            # Extract all contents to the specified path
            zf.extractall(extract_path)
        print(f"ZIP file downloaded from {url} and extracted to {extract_path}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
    except zipfile.BadZipFile:
        print("Error: The downloaded file is not a valid ZIP file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [4]:
from gutenbergpy.gutenbergcache import GutenbergCache, GutenbergCacheTypes
#for sqlite
# GutenbergCache.create()

In [5]:

#for sqlite
cache  = GutenbergCache.get_cache()

In [6]:
query = """
SELECT *
FROM books
INNER JOIN book_subjects bs ON books.id = bs.bookid
INNER JOIN subjects ON bs.subjectid = subjects.id
INNER JOIN languages ON languages.id = books.languageid
INNER JOIN titles ON books.id = titles.bookid
WHERE subjects.name LIKE "%Picture books for children%"
AND languages.name = 'en'
"""
cursor = cache.native_query(query)
books = cursor.fetchall()
# books

In [7]:
len(books)

68

In [8]:

for book in books:
    try:
        book_id = str(book[7])
        # print(book_id)
        url = index_url_base = f"http://aleph.gutenberg.org/{book_id[0]}/{book_id[1]}/{book_id[2]}/{book_id[3]}/{book_id}/{book_id}-h.zip"
        #my_req = requests.request("GET", url)
        print(url)
        download_and_extract_zip(url)
        base_name = url.split("/")[-1].split(".")[0]
        file_path = f"{base_name}/{base_name}.htm"
        with open(file_path, 'r', encoding="latin-1") as fp:
            soup = BeautifulSoup(fp, 'html.parser')
        for match in glob(base_name+"*"):
            if os.path.isdir(match):
                rmtree(match)
            else:
                os.remove(match)
                
    except:
        pass

http://aleph.gutenberg.org/1/0/6/3/10634/10634-h.zip
ZIP file downloaded from http://aleph.gutenberg.org/1/0/6/3/10634/10634-h.zip and extracted to .
http://aleph.gutenberg.org/7/4/1/8/74181/74181-h.zip
Error downloading the file: 404 Client Error: Not Found for url: http://aleph.gutenberg.org/7/4/1/8/74181/74181-h.zip
http://aleph.gutenberg.org/2/4/6/9/24694/24694-h.zip
ZIP file downloaded from http://aleph.gutenberg.org/2/4/6/9/24694/24694-h.zip and extracted to .
http://aleph.gutenberg.org/2/2/3/3/22335/22335-h.zip
ZIP file downloaded from http://aleph.gutenberg.org/2/2/3/3/22335/22335-h.zip and extracted to .
http://aleph.gutenberg.org/1/9/1/7/19177/19177-h.zip
ZIP file downloaded from http://aleph.gutenberg.org/1/9/1/7/19177/19177-h.zip and extracted to .
http://aleph.gutenberg.org/2/4/8/9/24894/24894-h.zip
ZIP file downloaded from http://aleph.gutenberg.org/2/4/8/9/24894/24894-h.zip and extracted to .
http://aleph.gutenberg.org/6/2/8/7/62877/62877-h.zip
Error downloading the file

In [ ]:

dc_subjects = {"Nursery rhymes",
               "Children's poetry",
               "Animals -- Juvenile fiction",
               "Chickens -- Juvenile fiction",
               "Fairy tales",
               "Children's stories",
               "Princesses -- Fiction",
               "Dogs -- Juvenile fiction",
               "Foxes -- Juvenile fiction",
               "Domestic animals -- Juvenile fiction",
               "Ducks -- Juvenile fiction",
               "Children's poetry, English",
               "Picture books for children",
               "Fairy tales -- Germany",
               "Pirates -- Fiction",
               "Fantasy literature",
               "Fairies -- Fiction"
               }


ZIP file downloaded from http://aleph.gutenberg.org/1/2/3/8/12383/12383-h.zip and extracted to .
ZIP file downloaded from http://aleph.gutenberg.org/1/0/0/0/10008/10008-h.zip and extracted to .
ZIP file downloaded from http://aleph.gutenberg.org/1/0/0/1/10018/10018-h/10018-h.zip and extracted to .
ZIP file downloaded from http://aleph.gutenberg.org/1/0/0/1/10018/10018-h.zip and extracted to .
ZIP file downloaded from http://aleph.gutenberg.org/1/0/0/3/10038/10038-h.zip and extracted to .
ZIP file downloaded from http://aleph.gutenberg.org/1/0/0/8/10084/10084-h.zip and extracted to .
ZIP file downloaded from http://aleph.gutenberg.org/1/0/1/1/10116/10116-h.zip and extracted to .
ZIP file downloaded from http://aleph.gutenberg.org/1/0/1/2/10120/10120-h.zip and extracted to .
ZIP file downloaded from http://aleph.gutenberg.org/1/0/1/2/10126/10126-h.zip and extracted to .
ZIP file downloaded from http://aleph.gutenberg.org/1/0/1/3/10138/10138-h.zip and extracted to .
ZIP file downloaded fr